# Cleaning Up Messy Data

### How do you know if a dataset is messy or not?

We're going to use a NYC 311 service request dataset in this Notebook. It's big and a bit unwieldy. This is a subset of the of 311 service requests from [NYC Open Data](https://nycopendata.socrata.com/Social-Services/311-Service-Requests-from-2010-to-Present/erm2-nwe9).

In [ ]:
%matplotlib inline

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Make the graphs a bit prettier, and bigger
plt.style.use('default')
plt.rcParams['figure.figsize'] = (15, 5)

In [ ]:
! head '311-service-requests.csv'

In the next cell, you'll probably see an error like "DtypeWarning: Columns (8) have mixed types". This means that it's encountered a problem reading in our data. In this case it almost certainly means that it has columns where some of the entries are strings and some are integers.

For now we're going to ignore it and hope we don't run into a problem, but in the long run we'd need to investigate this warning.

In [ ]:
requests = pd.read_csv('311-service-requests.csv')
requests.info()

In [ ]:
requests['Complaint Type']

In [ ]:
requests.head()

Looks bad - try to write some code to set these to NaN.

In [ ]:
requests['Complaint Type'].value_counts()

In [ ]:
complaint_counts = requests['Complaint Type'].value_counts()
complaint_counts[:10].plot(kind='bar');

### How do we know if it's messy? 

We're going to look at a few columns here. There are often problems with the zip code, so let's look at that first.
 
To get a sense for whether a column has problems, we often use `.unique()` to look at all its values. If it's a numeric column, we would plot a histogram to get a sense of the distribution.

When we look at the unique values in "Incident Zip", it quickly becomes clear that this is a mess.

Some of the problems:

* Some have been parsed as strings, and some as floats
* There are `NaN`s 
* Some of the zip codes are `29616-0759` or `83`
* There are some N/A values that pandas didn't recognize, like 'N/A' and 'NO CLUE'

What we can do:

* Normalize 'N/A' and 'NO CLUE' into regular NaN values
* Look at what's up with the 83, and decide what to do
* Make everything strings

In [ ]:
requests['Incident Zip'].unique()

### Fixing the nan values and string/float confusion

We can pass a `na_values` option to `pd.read_csv` to clean this up a little bit. We can also specify that the type of Incident Zip is a string, not a float.

In [ ]:
na_values = ['NO CLUE', 'N/A', '0']
requests = pd.read_csv('311-service-requests.csv', na_values=na_values, dtype={'Incident Zip': str})

In [ ]:
requests['Incident Zip'].unique()

### What's up with the dashes?

In [ ]:
rows_with_dashes = requests['Incident Zip'].str.contains('-').fillna(False)
len(requests[rows_with_dashes])

In [ ]:
requests[rows_with_dashes]

9-digit zip codes are normal. Let's look at all the zip codes with more than 5 digits, make sure they're okay, and then truncate them.

In [ ]:
long_zip_codes = requests['Incident Zip'].str.len() > 5
requests['Incident Zip'][long_zip_codes].unique()

Those all look okay to truncate.

In [ ]:
requests['Incident Zip'] = requests['Incident Zip'].str.slice(0, 5)

Done.

00083 seems like a broken zip code, but turns out Central Park's zip code 00083! The 00000 codes still seem suspicious. Let's look at them.

In [ ]:
requests[requests['Incident Zip'] == '00000']

Write some code to set 

In [ ]:
#zero_zips = requests['Incident Zip'] == '00000'
#requests['Incident Zip'][zero_zips] = np.nan

Great. Let's see where we are now:

In [ ]:
unique_zips = requests['Incident Zip'].unique()
#unique_zips.sort()
unique_zips

Amazing! This is much cleaner. There's something a bit weird here, though. For example, if you look up 77056 on Google maps, you'll see that it's in Texas.

Let's take a closer look:

In [ ]:
zips = requests['Incident Zip']
# Let's say the zips starting with '0' and '1' are okay, for now. (This isn't actually true - 13221 is in Syracuse.)
is_close = zips.str.startswith('0') | zips.str.startswith('1')
# There are a bunch of NaNs, but we're not interested in them right now, so we'll say they're False
is_far = ~(is_close) & zips.notnull()

In [ ]:
zips[is_far]

In [ ]:
requests[is_far][['Incident Zip', 'Descriptor', 'City']].sort_values('Incident Zip')

Okay, there really are requests coming from LA and Houston! Good to know. Filtering by zip code is probably a bad way to handle this. We should really be looking at the city instead.

In [ ]:
requests['City'].str.upper().value_counts()

It looks like these are legitimate complaints, so we'll just leave them alone.

### Putting it together

Here's what we ended up doing to clean up our zip codes, all together:

In [ ]:
na_values = ['NO CLUE', 'N/A', '0']
requests = pd.read_csv('311-service-requests.csv', 
                       na_values=na_values, 
                       dtype={'Incident Zip': str})

In [ ]:
def fix_zip_codes(zips):
    # Truncate everything to length 5 
    zips = zips.str.slice(0, 5)
    
    # Set 00000 zip codes to nan
    zero_zips = zips == '00000'
    zips[zero_zips] = np.nan
    
    return zips

In [ ]:
requests['Incident Zip'] = fix_zip_codes(requests['Incident Zip'])

In [ ]:
requests['Incident Zip'].unique()